Instructions/ Read me
1. Program starts by fetching data from given cnn url, two main dictionary is declared, one for stock with symbol as key(key-value pair) and one for stock details for the symbol (Price,open,volume).
2. First function fetches all the symbols from the a href tags and fills the main dictionary with symbol and name
3. Second function scraps all the data required from symbol and name stored in the main dictionary
4. Third function writes to csv file using the second dictionary (Excel file must not be open, in order for file to be generated)
5. Finally stock details are displayed and user is given a choice to enter a stock of their choice to see further details.
6. If given stock does not exist, exception is handled.
7. Between starting the program and the file being written, there's a small amount of wait period after which the user input in prompted since data has to be fetched, stored and written to file.

Reference used:
https://regex101.com/
https://towardsdatascience.com/web-scraping-with-python-beautifulsoup-40d2ce4b6252
https://www.datacamp.com/community/tutorials/amazon-web-scraping-using-beautifulsoup
https://stackabuse.com/reading-and-writing-csv-files-in-python/

In [2]:
import urllib.request, urllib.parse, urllib.error 
from bs4 import BeautifulSoup 
import ssl
import re
import csv

#Generating csv file with data from filevalue dictionary
def writeFile(category):
    i = 0
    j = 0
    #Existing file should not be opened, else file won't be written (filename - stocks.csv)
    with open('stocks.csv', 'w',newline='') as csvfile:
        #key list 
        excelColumns = ['Category', 'StockSymbol','Name','Opening','Previousclose','Volume','Market']
        writer = csv.DictWriter(csvfile, fieldnames=excelColumns)
        writer.writeheader()
        #Writing to file based on all our key values from stock details dictionary (fileValue)
        for link in fileValue:
            tempDict = stockDict[link]
            #Key - Column names, values - stock details 
            writer.writerow({'Category':stockCategory[j],'StockSymbol':tempDict['Link'],'Name':tempDict['Name'],'Opening':tempDict['Open'],'Previousclose':tempDict['PrevClose'],'Volume':tempDict['Volume'],'Market':tempDict['MarketCap']})
            i = i + 1
            if(i%10==0):
                j = j + 1         
    
#fetch stockdetails based on key values from stock Dictionary    
def appendData(url,pattern):
    #running the loop to fetch all details based on each key
    for key in stockDict.keys():
        uniqueKey = key
        tempDict = stockDict[uniqueKey]
        #key list for the main dictionary
        keyList = ['PrevClose','Open','Range','Volume','AvgVolume','MarketCap']
        url = "https://money.cnn.com/quote/quote.html?symb=" + uniqueKey
        html = urllib.request.urlopen(url)
        soup = BeautifulSoup(html, 'lxml')
        #find all the table data using table class
        rows = soup.find_all('table',{'class':"wsod_dataTable wsod_dataTableBig"})
        rowSet = rows[0].find_all('tr')
        #print(type(rowSet))
        #print("saraarar")
        i = 0
        #fetching all data using "td" and using regex to get the exact data needeed
        for row in rowSet:
            dataSet = row.find_all('td')
            dataSets = str(dataSet[1])
            tableValues = pattern
            tableValue = (re.sub(tableValues, '',dataSets))
            tempDict[keyList[i]] = tableValue
            i = i + 1
        stockDict[uniqueKey] = tempDict

def fetchFillData(data):
    i = 0
    j = 0
    for row in rows:
        #temporary dictionary to assign key with all details as value, key - stock, value - details
        dictTemp = dict()
        
        #find all the a href tags to get keys
        linkSet = row.find_all('a')
        #print("printing links",linkSet)
        #print(type(linkSet))
        links = str(linkSet)
        #print("printing string of links",links)
        #Using regex to get only the value we need
        regexPattern = re.compile('<.*?>')
        #print("printing clean links",regexPattern)
        stockValue = (re.sub(regexPattern, '',links))
        #print("printing clean links 2",stockValue)
        
        #Now to fetch all the value(details) for the same stock
        spanSet = row.find_all('span')
        #print("printing span",spanSet)
        spans = str(spanSet)
       # print("printing string of span",spans)
        #Using regex to get only the value we need
        spanValue = (re.sub(regexPattern, '',spans))
        #print("printing clean links 2 again?",spanValue)
        spanLength = len(spanValue)
        #Appending to dictionary (key,value - stock, details) and stock details 
        if(spanLength <= 2): continue
        spanList = spanValue.split(',')
        dictTemp['Link'] = stockValue[1:-1]
        dictTemp['Name'] = spanList[0][1:]
        stockDict[dictTemp['Link']] = dictTemp
        #print(dictTemp['Link']," ",dictTemp['Name'])
        if( i%10==0 ):
            print("\n")
            print(stockCategory[j])
            print("\n")
            j = j + 1
        print(dictTemp['Link'],": ",dictTemp['Name'])
        fileValue.append(dictTemp['Link'])
        i = i + 1
        #print("sarascvs",fileValue)
        

ctx = ssl.create_default_context()
ctx.check_hostname = False
ctx.verify_mode = ssl.CERT_NONE

url = "https://money.cnn.com/data/hotstocks/"
html = urllib.request.urlopen(url, context=ctx).read() 
soup = BeautifulSoup(html, 'html.parser')


#Get all tr rows from parsed HTML
rows = soup.find_all('tr')
#rows = set
#print(type(rows))
#Count of all tr rows found
rowsLength = len(rows)
#rowslength = 33
#print("rowsLength",rowsLength)

regexPattern = re.compile('<.*?>')
stockCategory = ['Most Active','Gainers','Losers']
stockDict = dict()
fileValue = list()
print("\n")
print("This is a program to scrape data from the https://money.cnn.com/data/hotstocks/  for a class project. ")
print("Which stock are you interested in: ")



#Write to two main dictinary, one to write to file, one to store key value pair of stock-details
fetchFillData(rows)
#print("saraFictr:",stockDict)
#print("saraFile:",fileValue)

#Now to get all data based on the key from stock dictionary
appendData(html,regexPattern)
#print("saraFictr:",stockDict)

#write to csv filewith keys as stock symbols stored in fileValue dictionary (31 records should be written to file)
writeFile(stockCategory)
print("\n")
print("stocks.csv file successfully written")
print("\n")


print("Categories")
print(stockCategory[0])
print(stockCategory[1])
print(stockCategory[2])

loopFlag = True
#print("sardicttionary",stockDict)
while loopFlag:
        userChoice = input("User inputs: ")
        if userChoice in stockDict:
            #print("key exists")
            loopFlag = False
            stockDetail = stockDict.get(userChoice)
            print("\n")
            #['PrevClose','Open','Range','Volume','AvgVolume','MarketCap']
            print("The data for ",userChoice," is the following: ")
            print("\n")
            print(stockDetail['Link'],stockDetail['Name'])
            print("OPEN: ", stockDetail['Open'])
            print("PREV CLOSE:  ",stockDetail['PrevClose'])
            print("VOLUME:",stockDetail['Volume'])
            print("MARKET CAP:",stockDetail['MarketCap'])
            print("\n")
            loopFlag = False
            break
        else:
            print("No such stock found, please try again")
            continue
            
print("Program successfully completed")



This is a program to scrape data from the https://money.cnn.com/data/hotstocks/  for a class project. 
Which stock are you interested in: 


Most Active


GE :  General Electric Co
PFE :  Pfizer Inc
T :  AT&amp;T Inc
F :  Ford Motor Co
CCL :  Carnival Corp
MRO :  Marathon Oil Corp
BAC :  Bank of America Corp
OXY :  Occidental Petroleum Corp
XOM :  Exxon Mobil Corp
WFC :  Wells Fargo &amp; Co


Gainers


EFX :  Equifax Inc
NCLH :  Norwegian Cruise Line Holdings Ltd
MRO :  Marathon Oil Corp
OXY :  Occidental Petroleum Corp
LUMN :  Centurylink Inc
HAL :  Halliburton Co
T :  AT&amp;T Inc
PXD :  Pioneer Natural Resources Co
XOM :  Exxon Mobil Corp
MTD :  Mettler-Toledo International Inc


Losers


LEN :  Lennar Corp
DHI :  D.R. Horton Inc
LEG :  Leggett &amp; Platt Inc
PHM :  Pultegroup Inc
NI :  NiSource Inc
WHR :  Whirlpool Corp
FLT :  Fleetcor Technologies Inc
AVB :  Avalonbay Communities Inc
KIM :  Kimco Realty Corp
D :  Dominion Energy Inc


stocks.csv file successfully written


Cat